In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [15]:
data=pd.read_csv('Dataset_Living_House - Dataset.csv')

In [16]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [17]:
features = ['society', 'location', 'total_sqft', 'surface_area', 'bedroom', 'bathroom', 'carport']

In [22]:
nlp_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('nlp', nlp_transformer, ['society']),
        ('num', numeric_transformer, numeric_features)
    ]
)

In [23]:
search_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('search', NearestNeighbors(n_neighbors=3, algorithm='ball_tree'))
])

In [24]:
search_model.fit(train_data[features])

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_base.py:564: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nlp',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['society']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['total_sqft', 'surface_area',
                                                   'bedroom', 'bathroom',
                                                   'carport'])])),
                ('search',
                 NearestNeighbors(algorithm='ball_tree', n_neighbors=3))])

In [25]:
def search_houses(query):
    # Transformasi query menggunakan preprocessor
    query_transformed = preprocessor.transform(pd.DataFrame([query], columns=features))
    
    # Temukan rumah terdekat berdasarkan query
    _, indices = search_model.named_steps['search'].kneighbors(query_transformed)

    # Ambil data rumah terdekat dari data latih
    result_houses = train_data.iloc[indices[0]][features]

    return result_houses

In [26]:
query = {'society': 'Michelia Randu', 'location': 'Babelan', 'price': 465000000, 'total_sqft': 36, 'surface_area': 60, 'bedroom': 2, 'bathroom': 1, 'carport': 2}
result = search_houses(query)

In [27]:
print("Hasil Pencarian:")
print(result)

Hasil Pencarian:
       society      location  total_sqft  surface_area  bedroom  bathroom  \
762  De Garden  Tambun Utara        55.0            60        2         2   
778  De Garden  Tambun Utara        55.0            60        2         2   
783  De Garden  Tambun Utara        55.0            60        2         2   

     carport  
762        2  
778        2  
783        2  
